### Import Necessary Libraries

In [13]:
import torch
import clip
import os
from PIL import Image
import numpy as np
from transformers import AdamW
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### Check for CUDA availability:
```python
device = "cuda" if torch.cuda.is_available() else "cpu" 
```  
This line checks if a CUDA-capable GPU is available. If it is, it sets the device variable to "cuda", otherwise it sets it to "cpu". This is important because using a GPU can significantly speed up the processing time compared to using the CPU.


### Load CLIP model and preprocessing:
```python
model, preprocess = clip.load("ViT-B/32", device=device)
```
This line loads the CLIP model ("ViT-B/32" version) and its associated preprocessing pipeline using the clip.load function. The device parameter ensures the model is loaded onto the appropriate device (GPU if available, otherwise CPU).

In [14]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

### Import images

In [15]:
# Directory containing the images
super_directory = r'resources\super'
sadyek_directory = r'resources\sadyek'

# Load super images in the directory
image_paths_super = [os.path.join(super_directory, file) for file in os.listdir(super_directory) if file.endswith('.bmp')]
images_super = [preprocess(Image.open(image_path)).unsqueeze(0).to(device) for image_path in image_paths_super]
# Load sadyek images in directory
image_paths_sadyek = [os.path.join(sadyek_directory, file) for file in os.listdir(sadyek_directory) if file.endswith('.bmp')]
images_sadyek = [preprocess(Image.open(image_path)).unsqueeze(0).to(device) for image_path in image_paths_sadyek]

### Combine images into a tensor

In [16]:
images_tensor_super = torch.cat(images_super, dim=0)

images_tensor_sadyek = torch.cat(images_sadyek, dim=0)

### Text Lable

In [17]:

text = clip.tokenize(['A dried fig with some opening parts that darker,revealing its interior,deformed,yammi'
                      , '  A dried fig have without opening parts,a little flat']).to(device)

In [18]:

with torch.no_grad():
    image_features = model.encode_image(images_tensor_super)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(images_tensor_super, text)
    probs_super = logits_per_image.softmax(dim=-1).cpu().numpy()


probs_super = (probs_super>0.5)    
Accuracy_super  = np.sum(probs_super[:,1])/len(probs_super)
print(f'Accuracy super : {Accuracy_super}')


with torch.no_grad():
    image_features = model.encode_image(images_tensor_sadyek)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(images_tensor_sadyek, text)
    probs_sadyek = logits_per_image.softmax(dim=-1).cpu().numpy()
    
probs_sadyek = (probs_sadyek>0.5)
Accuracy_sadyek = np.sum(probs_sadyek[:,0])/len(probs_sadyek)
print(f'Accuracy Sadyek : {Accuracy_sadyek}')





Accuracy super : 0.8316062176165803
Accuracy Sadyek : 0.8337349397590361
